In [64]:
import pandas as pd
import requests
import csv
from bs4 import BeautifulSoup

In [93]:
Member_URLS = []

base_url = 'https://www.gabar.org/membersearchresults.cfm?start={}&id={}'
pages = [1, 26, 51]
id = 'A06532F0-E305-D491-EFFB8852D50B8F99'

for page in pages:
    url = base_url.format(page, id)
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')
    member_names = soup.find_all('div', {'class': 'member-name'})

    for member_name in member_names:
        links = member_name.find_all('a')
        for link in links:
            print(link.get('href'))
            Member_URLS.append(f"https://www.gabar.org"+str(link.get('href')))
print(Member_URLS)

[]


In [98]:
#get urls of member profiles
Member_URLS = []


for page_num in range(1, 2):
    url = f'https://www.gabar.org/membersearchresults.cfm?page={page_num}'
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')
    member_names = soup.find_all('div', {'class': 'member-name'})

    for member_name in member_names:
        links = member_name.find_all('a')
        for link in links:
            Member_URLS.append(f"https://www.gabar.org"+str(link.get('href')))

print(Member_URLS)

['https://www.gabar.org/MemberSearchDetail.cfm?ID=Njc0MTg3', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDMx', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=ODkwMTE2', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDM1', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDUw', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MzMzOTI0', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDU1', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDc1', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDc5', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDg3', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MTQyMjEw', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MjYyMjEx', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDc2MTEx', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MDAwMDg5', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MjExNTg5', 'https://www.gabar.org/MemberSearchDetail.cfm?ID=MzE1MTYx', 'https://www.gabar.org/MemberSearchDeta

In [65]:
#get urls of member profiles
Member_URLS = []


for page_num in range(1, 2):
    url = f'https://www.gabar.org/membersearchresults.cfm?page={page_num}'
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')
    member_names = soup.find_all('div', {'class': 'member-name'})

    for member_name in member_names:
        links = member_name.find_all('a')
        for link in links:
            Member_URLS.append(f"https://www.gabar.org"+str(link.get('href')))

#use the mmber profile links to scrape the contact infor for each member
contact_info = []

for url in Member_URLS:
    # Send a GET request to the URL and retrieve the HTML content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        # Find all the divs with class 'course-box'
        course_boxes = soup.find_all('div', {'class': 'course-box'})
        course_ids = soup.find('div', {'class': 'course-id'})

        name = (course_ids.get_text())
    except:
        None

    # Loop through each course box and extract the relevant information
    email_list = []
    try:
        course_boxes = soup.find_all('div', {'class': 'course-box'})

        for box in course_boxes:
            td_tags = box.find_all('td')
            for td in td_tags:
                email_link = td.find('a', href=True)
                if email_link and email_link['href'].startswith('mailto:'):
                    email_list.append(email_link['href'][7:])

        email = (email_list[0])

        # Create a dictionary to store the course information
        course = {
            'name': name,
            'email': email,
        }

        # Add the course information to the list
        contact_info.append(course)

    except:
        None

# Open a new file in "write" mode and specify the delimiter as a comma
with open('contact_info.csv', mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')

    # Write the header row
    writer.writerow(['Name', 'Email'])

    # Iterate over each dictionary in the list and write a new row for each
    for item in contact_info:
        writer.writerow([item['name'], item['email']])

# Convert the list of courses to a CSV
# df = pd.DataFrame(contact_info)

# # Print the DataFrame
# print(contact_info)
# display(df)

# df.to_csv('contact_info.csv', index=False)

In [112]:
import requests
from bs4 import BeautifulSoup

# create an empty list to store the LegalProfile URLs
legal_profile_urls = []

# specify the range of pages to scrape
start_page = 0
end_page = 1

# loop through the pages in the specified range
for page_number in range(start_page, end_page+1):
    # make a request to the current page
    url = f'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory.aspx?ShowSearchResults=TRUE&Country=USA&Page={page_number}'
    response = requests.get(url)
    
    # create a BeautifulSoup object from the response content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # find all the <tr> elements with class "grid-row"
    rows = soup.find_all('tr', {'class': 'grid-row'})
    
    # loop through the rows and extract the LegalProfile URL from the onclick attribute
    for row in rows:
        onclick = row.get('onclick')
        if onclick:
            url = onclick.split("'")[1]
            legal_profile_urls.append(f"https://www.mywsba.org/PersonifyEbusiness/"+str(url))

# print the list of LegalProfile URLs
len(legal_profile_urls)

['https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000000000001', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000000000497', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000000000002', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000009101360', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000000000003', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000009101361', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000000000004', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000009101362', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000000000005', 'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory/LegalProfile.aspx?Usr_ID=000009062720', 'https://www.mywsba

40

In [140]:
import requests
from bs4 import BeautifulSoup
import csv


# create an empty list to store the LegalProfile URLs
legal_profile_urls = []

# specify the range of pages to scrape there are 1734 pages
start_page = 1301
end_page = 1734

# loop through the pages in the specified range
for page_number in range(start_page, end_page+1):
    # make a request to the current page
    url = f'https://www.mywsba.org/PersonifyEbusiness/LegalDirectory.aspx?ShowSearchResults=TRUE&EligibleToPractice=Y&Country=USA&Page={page_number}'
    response = requests.get(url)
    
    # create a BeautifulSoup object from the response content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # find all the <tr> elements with class "grid-row"
    rows = soup.find_all('tr', {'class': 'grid-row'})
    
    # loop through the rows and extract the LegalProfile URL from the onclick attribute
    for row in rows:
        onclick = row.get('onclick')
        if onclick:
            url = onclick.split("'")[1]
            legal_profile_urls.append(f"https://www.mywsba.org/PersonifyEbusiness/"+str(url))

    print(page_number)

# initialize empty list to store scraped data
data = []

# loop through each URL and scrape the data
for url in legal_profile_urls:
    # make a GET request to the URL
    response = requests.get(url)


    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the relevant elements by ID and extract the text
    member_no = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblMemberNo'}).text
    license_type = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblLicenseType'}).text
    eligible_to_practice = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblEligibleToPractice'}).text
    status = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblStatus'}).text
    wa_admit_date = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblWaAdmitDate'}).text
    email = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblEmail'}).text
    member_name = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblMemberName'}).text
    address = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblAddress'}).text
    phone = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblPhone'}).text
    area_of_practice = soup.find('span', {'id': 'dnn_ctr2977_DNNWebControlContainer_ctl00_lblPracticeAreas'}).text

    # append the data to the list
    data.append([member_name, member_no, license_type, eligible_to_practice, status, wa_admit_date, email, address, phone, area_of_practice])
    print(url)
    
# write the data to a CSV file
#with open('Washington_Bar_Association.csv', 'w', newline='') as file:
with open('Washington_Bar_Association.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Full_Name','Member_No.', 'License_Type', 'Eligible_to_Practice', 'Status', 'WSBA_Admit_Date', 'Email','address','phone','legal_speciality'])
    writer.writerows(data)

1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400
1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472
1473
1474
1475
1476
1477
1478
1479
1480
1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494
1495
1496
1497
1498
1499
1500
